In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('pf_DHODH_03_bioactivity_data_curated.csv')
df

,molecule_chembl_id,canonical_smiles,standard_value,class
0,CHEMBL199572,CN(C(=O)c1ccc(-c2ccccc2)cc1)c1ccccc1C(=O)O,42600.0,inactive
1,CHEMBL199574,O=C(Nc1ccccc1C(=O)O)c1ccc2cc(Br)ccc2c1,142600.0,inactive
2,CHEMBL372561,CN(C(=O)c1ccc2cc(Br)ccc2c1)c1ccccc1C(=O)O,93400.0,inactive
3,CHEMBL370865,O=C(Nc1ccccc1C(=O)O)c1ccc(-c2ccccc2)cc1,153500.0,inactive
4,CHEMBL199575,CN(C(=O)c1ccc2ccccc2c1)c1ccccc1C(=O)O,200000.0,inactive
...,...,...,...,...
546,CHEMBL4544292,CCOc1nn(C)c(COc2ccccc2)c1C(=O)O,250000.0,inactive
547,CHEMBL4569109,Cn1nc(OCC2CC2)c(C(=O)O)c1COc1ccccc1,250000.0,inactive
548,CHEMBL4568957,Cn1nc(OCc2ccccc2)c(C(=O)O)c1COc1ccccc1,250000.0,inactive
549,CHEMBL4449622,Cn1nc(O)c(C(N)=O)c1COc1ccccc1,250000.0,inactive


In [3]:
df_no_smiles = df.drop(columns='canonical_smiles')

In [4]:
smiles = []

for i in df.canonical_smiles.tolist():
  cpd = str(i).split('.')
  cpd_longest = max(cpd, key = len)
  smiles.append(cpd_longest)

smiles = pd.Series(smiles, name = 'canonical_smiles')

In [5]:
df_clean_smiles = pd.concat([df_no_smiles,smiles], axis=1)
df_clean_smiles

,molecule_chembl_id,standard_value,class,canonical_smiles
0,CHEMBL199572,42600.0,inactive,CN(C(=O)c1ccc(-c2ccccc2)cc1)c1ccccc1C(=O)O
1,CHEMBL199574,142600.0,inactive,O=C(Nc1ccccc1C(=O)O)c1ccc2cc(Br)ccc2c1
2,CHEMBL372561,93400.0,inactive,CN(C(=O)c1ccc2cc(Br)ccc2c1)c1ccccc1C(=O)O
3,CHEMBL370865,153500.0,inactive,O=C(Nc1ccccc1C(=O)O)c1ccc(-c2ccccc2)cc1
4,CHEMBL199575,200000.0,inactive,CN(C(=O)c1ccc2ccccc2c1)c1ccccc1C(=O)O
...,...,...,...,...
546,CHEMBL4544292,250000.0,inactive,CCOc1nn(C)c(COc2ccccc2)c1C(=O)O
547,CHEMBL4569109,250000.0,inactive,Cn1nc(OCC2CC2)c(C(=O)O)c1COc1ccccc1
548,CHEMBL4568957,250000.0,inactive,Cn1nc(OCc2ccccc2)c(C(=O)O)c1COc1ccccc1
549,CHEMBL4449622,250000.0,inactive,Cn1nc(O)c(C(N)=O)c1COc1ccccc1


#### Calculate Lipinski descriptors
Lipinski's Rule of Five is a guideline used in drug discovery to assess whether a compound is likely to be orally bioavailable, meaning it can be absorbed into the bloodstream when taken orally. The rule states that a compound is more likely to be orally bioavailable if it meets the following criteria: 1)
Molecular Weight: The compound's molecular weight should be less than 500 Dalto 2) 

Hydrogen Bond Donors: The compound should have no more than 5 hydrogen bond rs 3) no

Hydrogen Bond Acceptors: The compound should have no more than 10 hydrogen bondtercc 4) p

s.
Lipophilicity (LogP): The compound's logarithm of the partition coefficient (LogP) should be less an 5.

If a compound meets these criteria, it is considered more likely to have favorable oral bioavailability characteristics, which is an important factor in drug veopment. .

In [6]:
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski

In [7]:
def lipinski(smiles, verbose=False):

    moldata= []
    for elem in smiles:
        mol=Chem.MolFromSmiles(elem) 
        moldata.append(mol)
       
    baseData= np.arange(1,1)
    i=0  
    for mol in moldata:        
       
        desc_MolWt = Descriptors.MolWt(mol)
        desc_MolLogP = Descriptors.MolLogP(mol)
        desc_NumHDonors = Lipinski.NumHDonors(mol)
        desc_NumHAcceptors = Lipinski.NumHAcceptors(mol)
           
        row = np.array([desc_MolWt,
                        desc_MolLogP,
                        desc_NumHDonors,
                        desc_NumHAcceptors])   
    
        if(i==0):
            baseData=row
        else:
            baseData=np.vstack([baseData, row])
        i=i+1      
    
    columnNames=["MW","LogP","NumHDonors","NumHAcceptors"]   
    descriptors = pd.DataFrame(data=baseData,columns=columnNames)
    
    return descriptors

In [8]:
df_lipinski = lipinski(df_clean_smiles.canonical_smiles)
df_lipinski
     

,MW,LogP,NumHDonors,NumHAcceptors
0,331.371,4.32840,1.0,2.0
1,370.202,4.55280,2.0,2.0
2,384.229,4.57710,1.0,2.0
3,317.344,4.30410,2.0,2.0
4,305.333,3.81460,1.0,2.0
...,...,...,...,...
546,276.292,2.09600,1.0,5.0
547,302.330,2.48610,1.0,5.0
548,338.363,3.27630,1.0,5.0
549,247.254,0.80360,2.0,5.0


In [9]:
df

,molecule_chembl_id,canonical_smiles,standard_value,class
0,CHEMBL199572,CN(C(=O)c1ccc(-c2ccccc2)cc1)c1ccccc1C(=O)O,42600.0,inactive
1,CHEMBL199574,O=C(Nc1ccccc1C(=O)O)c1ccc2cc(Br)ccc2c1,142600.0,inactive
2,CHEMBL372561,CN(C(=O)c1ccc2cc(Br)ccc2c1)c1ccccc1C(=O)O,93400.0,inactive
3,CHEMBL370865,O=C(Nc1ccccc1C(=O)O)c1ccc(-c2ccccc2)cc1,153500.0,inactive
4,CHEMBL199575,CN(C(=O)c1ccc2ccccc2c1)c1ccccc1C(=O)O,200000.0,inactive
...,...,...,...,...
546,CHEMBL4544292,CCOc1nn(C)c(COc2ccccc2)c1C(=O)O,250000.0,inactive
547,CHEMBL4569109,Cn1nc(OCC2CC2)c(C(=O)O)c1COc1ccccc1,250000.0,inactive
548,CHEMBL4568957,Cn1nc(OCc2ccccc2)c(C(=O)O)c1COc1ccccc1,250000.0,inactive
549,CHEMBL4449622,Cn1nc(O)c(C(N)=O)c1COc1ccccc1,250000.0,inactive


#### Now, let's combine the 2 DataFrame

In [10]:
df_combined = pd.concat([df,df_lipinski], axis=1)

In [11]:
df_combined

,molecule_chembl_id,canonical_smiles,standard_value,class,MW,LogP,NumHDonors,NumHAcceptors
0,CHEMBL199572,CN(C(=O)c1ccc(-c2ccccc2)cc1)c1ccccc1C(=O)O,42600.0,inactive,331.371,4.32840,1.0,2.0
1,CHEMBL199574,O=C(Nc1ccccc1C(=O)O)c1ccc2cc(Br)ccc2c1,142600.0,inactive,370.202,4.55280,2.0,2.0
2,CHEMBL372561,CN(C(=O)c1ccc2cc(Br)ccc2c1)c1ccccc1C(=O)O,93400.0,inactive,384.229,4.57710,1.0,2.0
3,CHEMBL370865,O=C(Nc1ccccc1C(=O)O)c1ccc(-c2ccccc2)cc1,153500.0,inactive,317.344,4.30410,2.0,2.0
4,CHEMBL199575,CN(C(=O)c1ccc2ccccc2c1)c1ccccc1C(=O)O,200000.0,inactive,305.333,3.81460,1.0,2.0
...,...,...,...,...,...,...,...,...
546,CHEMBL4544292,CCOc1nn(C)c(COc2ccccc2)c1C(=O)O,250000.0,inactive,276.292,2.09600,1.0,5.0
547,CHEMBL4569109,Cn1nc(OCC2CC2)c(C(=O)O)c1COc1ccccc1,250000.0,inactive,302.330,2.48610,1.0,5.0
548,CHEMBL4568957,Cn1nc(OCc2ccccc2)c(C(=O)O)c1COc1ccccc1,250000.0,inactive,338.363,3.27630,1.0,5.0
549,CHEMBL4449622,Cn1nc(O)c(C(N)=O)c1COc1ccccc1,250000.0,inactive,247.254,0.80360,2.0,5.0


#### Standardize and convert IC50 values to pIC50 values

To improve the model's ability to predict standard values, we can convert IC50 values to pIC50 values using the formula: −log(IC50×10−9). This transformation is often used in pharmacology to express IC50 values in a logarithmic scale, making them more suitable for analysis and modeling

In [26]:
import numpy as np

def pIC50(input):
    pIC50 = []

    for i in input['standard_value_norm']:
        molar = i*(10**-9) # Converts nM to M
        pIC50.append(-np.log10(molar))

    input['pIC50'] = pIC50
    x = input.drop('standard_value_norm', axis=1)
        
    return x
     

##### Point to note: Values greater than 100,000,000 will be fixed at 100,000,000 otherwise the negative logarithmic value will become negative.

In [21]:
df_combined.standard_value.describe()

count       551.000000
mean      33974.369383
std       82621.937247
min           6.000000
25%         327.000000
50%        7000.000000
75%       27490.000000
max      870963.590000
Name: standard_value, dtype: float64

In [22]:
-np.log10( (10**-9)* 100000000 )

1.0

In [23]:
-np.log10( (10**-9)* 10000000000 )

-1.0

##### We will first apply the norm_value() function so that the values in the standard_value column is normalized.

In [27]:
def norm_value(input):
    norm = []

    for i in input['standard_value']:
        if i > 100000000:
          i = 100000000
        norm.append(i)

    input['standard_value_norm'] = norm
    x = input.drop('standard_value',axis=1)
        
    return x
     

In [28]:
df_norm = norm_value(df_combined)
df_norm     

,molecule_chembl_id,canonical_smiles,class,MW,LogP,NumHDonors,NumHAcceptors,standard_value_norm
0,CHEMBL199572,CN(C(=O)c1ccc(-c2ccccc2)cc1)c1ccccc1C(=O)O,inactive,331.371,4.32840,1.0,2.0,42600.0
1,CHEMBL199574,O=C(Nc1ccccc1C(=O)O)c1ccc2cc(Br)ccc2c1,inactive,370.202,4.55280,2.0,2.0,142600.0
2,CHEMBL372561,CN(C(=O)c1ccc2cc(Br)ccc2c1)c1ccccc1C(=O)O,inactive,384.229,4.57710,1.0,2.0,93400.0
3,CHEMBL370865,O=C(Nc1ccccc1C(=O)O)c1ccc(-c2ccccc2)cc1,inactive,317.344,4.30410,2.0,2.0,153500.0
4,CHEMBL199575,CN(C(=O)c1ccc2ccccc2c1)c1ccccc1C(=O)O,inactive,305.333,3.81460,1.0,2.0,200000.0
...,...,...,...,...,...,...,...,...
546,CHEMBL4544292,CCOc1nn(C)c(COc2ccccc2)c1C(=O)O,inactive,276.292,2.09600,1.0,5.0,250000.0
547,CHEMBL4569109,Cn1nc(OCC2CC2)c(C(=O)O)c1COc1ccccc1,inactive,302.330,2.48610,1.0,5.0,250000.0
548,CHEMBL4568957,Cn1nc(OCc2ccccc2)c(C(=O)O)c1COc1ccccc1,inactive,338.363,3.27630,1.0,5.0,250000.0
549,CHEMBL4449622,Cn1nc(O)c(C(N)=O)c1COc1ccccc1,inactive,247.254,0.80360,2.0,5.0,250000.0


In [29]:

df_norm.standard_value_norm.describe()

count       551.000000
mean      33974.369383
std       82621.937247
min           6.000000
25%         327.000000
50%        7000.000000
75%       27490.000000
max      870963.590000
Name: standard_value_norm, dtype: float64

In [34]:
import numpy as np

def pIC50(input):
    pIC50 = []

    for i in input['standard_value_norm']:
        molar = i*(10**-9) # Converts nM to M
        pIC50.append(-np.log10(molar))

    input['pIC50'] = pIC50
    x = input.drop('standard_value_norm', axis=1)
        
    return x

In [35]:
df_final = pIC50(df_norm)
df_final     

,molecule_chembl_id,canonical_smiles,class,MW,LogP,NumHDonors,NumHAcceptors,pIC50
0,CHEMBL199572,CN(C(=O)c1ccc(-c2ccccc2)cc1)c1ccccc1C(=O)O,inactive,331.371,4.32840,1.0,2.0,4.370590
1,CHEMBL199574,O=C(Nc1ccccc1C(=O)O)c1ccc2cc(Br)ccc2c1,inactive,370.202,4.55280,2.0,2.0,3.845880
2,CHEMBL372561,CN(C(=O)c1ccc2cc(Br)ccc2c1)c1ccccc1C(=O)O,inactive,384.229,4.57710,1.0,2.0,4.029653
3,CHEMBL370865,O=C(Nc1ccccc1C(=O)O)c1ccc(-c2ccccc2)cc1,inactive,317.344,4.30410,2.0,2.0,3.813892
4,CHEMBL199575,CN(C(=O)c1ccc2ccccc2c1)c1ccccc1C(=O)O,inactive,305.333,3.81460,1.0,2.0,3.698970
...,...,...,...,...,...,...,...,...
546,CHEMBL4544292,CCOc1nn(C)c(COc2ccccc2)c1C(=O)O,inactive,276.292,2.09600,1.0,5.0,3.602060
547,CHEMBL4569109,Cn1nc(OCC2CC2)c(C(=O)O)c1COc1ccccc1,inactive,302.330,2.48610,1.0,5.0,3.602060
548,CHEMBL4568957,Cn1nc(OCc2ccccc2)c(C(=O)O)c1COc1ccccc1,inactive,338.363,3.27630,1.0,5.0,3.602060
549,CHEMBL4449622,Cn1nc(O)c(C(N)=O)c1COc1ccccc1,inactive,247.254,0.80360,2.0,5.0,3.602060


In [36]:
df_final.pIC50.describe()

count    551.000000
mean       5.494939
std        1.165906
min        3.060000
25%        4.560849
50%        5.154902
75%        6.485471
max        8.221849
Name: pIC50, dtype: float64

In [37]:
df_final.to_csv('pf_DHODH_04_bioactivity_data_3class_pIC50.csv')